# Pruning with our automatic structured Pruning framework
Welcome to an end-to-end example for magnitude-based weight pruning

**Summary**

In this tutorial, you will:

* Train a tf.keras model for CIFAR10 from scratch.
* Fine tune the model by applying the pruning Framework and see the accuracy.

If you want to execute this notebook in Google Colab, uncomment the code below.

In [1]:
# !git clone https://github.com/Hahn-Schickard/Automatic-Structured-Pruning
# !echo $CWD
# !cp -rf /content/Automatic-Structured-Pruning/* /content/

In [2]:
import pruning

## Train a model for CIFAR10 without pruning
Download and prepare the CIFAR10 dataset.
The CIFAR10 dataset contains 60,000 color images in 10 classes, with 6,000 images in each class. The dataset is divided into 50,000 training images and 10,000 testing images. The classes are mutually exclusive and there is no overlap between them.

Create the convolutional base
The 6 lines of code below define the convolutional base using a common pattern: a stack of Conv2D and MaxPooling2D layers.

As input, a CNN takes tensors of shape (image_height, image_width, color_channels), ignoring the batch size. If you are new to these dimensions, color_channels refers to (R,G,B). In this example, you will configure our CNN to process inputs of shape (32, 32, 3), which is the format of CIFAR images. You can do this by passing the argument input_shape to our first layer.

To complete our model, you will feed the last output tensor from the convolutional base (of shape (4, 4, 64)) into one or more Dense layers to perform classification. Dense layers take vectors as input (which are 1D), while the current output is a 3D tensor. First, you will flatten (or unroll) the 3D output to 1D, then add one or more Dense layers on top. CIFAR has 10 output classes, so you use a final Dense layer with 10 outputs and a softmax activation.

In [3]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models


(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dropout(0.25))
model.add(layers.Dense(10, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 64)          36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 2, 2, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 256)               0

Above, you can see that the output of every Conv2D and MaxPooling2D layer is a 3D tensor of shape (height, width, channels). The width and height dimensions tend to shrink as you go deeper in the network. The number of output channels for each Conv2D layer is controlled by the first argument (e.g., 32 or 64). Typically, as the width and height shrink, you can afford (computationally) to add more output channels in each Conv2D layer.
As you can see, our (4, 4, 64) outputs were flattened into vectors of shape (1024) before going through two Dense layers.

## Compile and train the model

In [4]:

comp = {
"optimizer":'adam',
"loss": tf.keras.losses.SparseCategoricalCrossentropy(),
"metrics": ['accuracy']}

model.compile(**comp)
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)]

history = model.fit(train_images, train_labels, validation_split=0.2, epochs=30, batch_size=128, callbacks=callbacks)

test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)

Epoch 1/30
313/313 [==============================] - 5s 11ms/step - loss: 1.9466 - accuracy: 0.2650 - val_loss: 1.6042 - val_accuracy: 0.4069
Epoch 2/30
313/313 [==============================] - 3s 9ms/step - loss: 1.6038 - accuracy: 0.4033 - val_loss: 1.4802 - val_accuracy: 0.4735
Epoch 3/30
313/313 [==============================] - 3s 9ms/step - loss: 1.4489 - accuracy: 0.4751 - val_loss: 1.2951 - val_accuracy: 0.5372
Epoch 4/30
313/313 [==============================] - 3s 9ms/step - loss: 1.3398 - accuracy: 0.5192 - val_loss: 1.2103 - val_accuracy: 0.5653
Epoch 5/30
313/313 [==============================] - 3s 9ms/step - loss: 1.2698 - accuracy: 0.5511 - val_loss: 1.1385 - val_accuracy: 0.5972
Epoch 6/30
313/313 [==============================] - 3s 9ms/step - loss: 1.2067 - accuracy: 0.5747 - val_loss: 1.1001 - val_accuracy: 0.6162
Epoch 7/30
313/313 [==============================] - 3s 9ms/step - loss: 1.1566 - accuracy: 0.5928 - val_loss: 1.0455 - val_accuracy: 0.6291
Epoch

In [5]:
model.save('CIFAR10_model.h5') 

## Fine-tune pre-trained model with pruning
You will apply pruning to the whole model and see this in the model summary.

In this example, you prune the model with 30% dense pruning and 40% filter pruning.

In [6]:
dense_prune_rate=30
conv_prune_rate=40
prunemodel=pruning.prune_model('./CIFAR10_model.h5', dense_prune_rate, conv_prune_rate,'L2', num_classes=10)

Finish with pruning
Conv2D
No Dense or Conv2D
Conv2D
No Dense or Conv2D
Conv2D
No Dense or Conv2D
No Dense or Conv2D
Dense
No Dense or Conv2D
Dense
No Dense or Conv2D
Before pruning:
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 64)          36928     
_________________________________________________________________
max_p

d:\Users\Danie\Desktop\Hahn-Schickard\Automatic-Structured-Pruning\pruning.py:131: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return (np.array(layer_types), np.array(layer_params), layer_output_shape,


We see how we get less parameter in the pruned model.

## Compile and re-train the model

In [7]:
prunemodel.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 20)        560       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 20)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 39)        7059      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 39)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 39)          13728     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 2, 2, 39)          0         
_________________________________________________________________
flatten (Flatten)            (None, 156)               0

In [8]:
comp = {
"optimizer":'adam',
"loss": tf.keras.losses.SparseCategoricalCrossentropy(),
"metrics": ['accuracy']}

prunemodel.compile(**comp)

history = prunemodel.fit(train_images, train_labels, epochs=10, validation_split=0.2)

Epoch 1/10
1250/1250 [==============================] - 8s 6ms/step - loss: 1.4179 - accuracy: 0.4998 - val_loss: 1.1750 - val_accuracy: 0.5883
Epoch 2/10
1250/1250 [==============================] - 7s 6ms/step - loss: 1.1694 - accuracy: 0.5997 - val_loss: 1.0824 - val_accuracy: 0.6200
Epoch 3/10
1250/1250 [==============================] - 7s 6ms/step - loss: 1.0957 - accuracy: 0.6254 - val_loss: 1.0273 - val_accuracy: 0.6477
Epoch 4/10
1250/1250 [==============================] - 7s 6ms/step - loss: 1.0391 - accuracy: 0.6468 - val_loss: 0.9706 - val_accuracy: 0.6674
Epoch 5/10
1250/1250 [==============================] - 7s 6ms/step - loss: 1.0001 - accuracy: 0.6605 - val_loss: 0.9864 - val_accuracy: 0.6621
Epoch 6/10
1250/1250 [==============================] - 7s 6ms/step - loss: 0.9784 - accuracy: 0.6700 - val_loss: 0.9403 - val_accuracy: 0.6750
Epoch 7/10
1250/1250 [==============================] - 7s 6ms/step - loss: 0.9502 - accuracy: 0.6804 - val_loss: 0.9211 - val_accuracy:

Compare both models

In [9]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
test_loss, test_acc = prunemodel.evaluate(test_images,  test_labels, verbose=2)

313/313 - 1s - loss: 0.8825 - accuracy: 0.6915
313/313 - 1s - loss: 0.9339 - accuracy: 0.6808


# Prune a model to a maximum accuracy loss

We define the arguments to compile the model. In this case, we only want to have an accuracy loss of 3%.
In this example we have loaded the data directly from a TensorFlow dataset. Therefore, we do not have a defined dataloader (path or file). However, the structure of the training data is the same as the data we would read from a Python file. Therefore, we use a trick here and pass an existing FILE from the current folder as the dataloader path. This way the correct functions will be executed afterwards and no error will be issued.

In [10]:
comp = {
  "optimizer": 'adam',
  "loss": tf.keras.losses.SparseCategoricalCrossentropy(),
  "metrics": 'accuracy'
}

automodel = pruning.pruning_for_acc('./CIFAR10_model.h5', train_images, train_labels, comp, pruning_acc=None,
                                  max_acc_loss=3, num_classes=10, label_one_hot=False, data_loader_path="./CIFAR10_model.h5")

313/313 [==============================] - 2s 4ms/step - loss: 0.7259 - accuracy: 0.7516
Start model accuracy: 75.16000270843506%
Next pruning factors: 5
Finish with pruning
Conv2D
No Dense or Conv2D
Conv2D
No Dense or Conv2D
Conv2D
No Dense or Conv2D
No Dense or Conv2D
Dense
No Dense or Conv2D
Dense
No Dense or Conv2D
Before pruning:
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________

In [11]:
automodel.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 64)          36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 2, 2, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 256)               0

Compare both models

In [12]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
test_loss, test_acc = automodel.evaluate(test_images,  test_labels, verbose=2)

313/313 - 1s - loss: 0.8825 - accuracy: 0.6915
313/313 - 1s - loss: 0.8825 - accuracy: 0.6915
